# Assume summary_df exists from previous code
main_country = "Philippines"
compare_country = "Japan"

def generate_md_report(df, main_country, compare_country, filename="debt_report.md"):
    md_lines = []

    # Title
    md_lines.append("# National Debt Comparison Report\n")

    # Country Profiles
    md_lines.append("## Country Profiles\n")
    for country in [main_country, compare_country]:
        debt = df.loc[df['Metric']=="National Debt (USD)", country].values[0]
        pct_gdp = df.loc[df['Metric']=="Debt % GDP", country].values[0]
        per_capita = df.loc[df['Metric']=="Debt Per Capita", country].values[0]
        md_lines.append(f"**{country}** has a national debt of **{debt}**, which is **{pct_gdp}** of its GDP, and a per capita debt of **{per_capita}**.\n")

    # Comparative Analysis
    md_lines.append("---\n")
    md_lines.append("## Comparative Analysis\n")
    for metric in df['Metric']:
        val_main = df.loc[df['Metric']==metric, main_country].values[0]
        val_other = df.loc[df['Metric']==metric, compare_country].values[0]
        ratio_main = df.loc[df['Metric']==metric, "Ratio (main/other)"].values[0]
        ratio_inverse = df.loc[df['Metric']==metric, "Ratio (other/main)"].values[0]

        md_lines.append(f"**{metric}**  ")
        md_lines.append(f"*{main_country}* has **{val_main}**, which is **{ratio_main}×** of *{compare_country}*'s **{val_other}**.  ")
        md_lines.append(f"On the other hand, *{compare_country}* is **{ratio_inverse}× bigger** than *{main_country}*.\n")

    # Summary
    md_lines.append("---\n")
    md_lines.append("### Summary\n")
    md_lines.append(f"From the comparison above, it is clear that while the *{main_country}* has a significantly smaller total national debt than *{compare_country}*, the differences are substantial both in terms of GDP percentage and per capita debt. The ratios indicate that *{compare_country}*'s debt metrics are multiple times larger across all three measures.\n")

    # Write to file
    with open(filename, "w") as f:
        f.write("\n".join(md_lines))

    print(f"Markdown report saved as '{filename}'")

# Usage
generate_md_report(summary_df, main_country, compare_country)
